In [ ]:
# You may first need to open your terminal and install jsonlines by typing:
# "pip install jsonlines"

In [ ]:
import requests
import json
import jsonlines

# Change these: Add your credentials

In [ ]:
USER = ''
PASS = ''
HOST = 'http://gwaspace-test.wrlc.org:8089' #don't end with a slash
def aspace_auth(host, username, password):
    auth = requests.post(host + '/users/' + username + '/login',
                        params={'password' : password})
    if auth.status_code == 200:
        token = auth.json()['session']
        headers = {'X-ArchivesSpace-Session': token}
        return(headers)
    else:
        return(False)

headers = aspace_auth(HOST, USER, PASS)
print(headers)

# Don't need to change anything below this comment (in theory . . . )

In [ ]:
# We need a metadata field that's always empty, where we can "park" a bit of information. 

# Let's check to see if the "ils_holding_id" is always empty.

print('Checking for items using the ils_holding_id field. . . ')
      
with jsonlines.open('nea_containers.jsonl') as dup_list:
    for container in dup_list:
        # Save the container's box number and URL to variables.
        uri = (container['uri'])
        # Get a fresh copy of the container from ASpace
        container = requests.get(HOST + uri,
                                  headers=headers)
        container = container.json()
        if container.get('ils_holding_id', None) is None:
            pass
        else:
            print("Oops! " + uri + " has ils_holding_id: " + container['ils_holding_id'])
            print(container.keys())
print("\nAll done checking!")

In [ ]:
# Make a blank list. 
# As the script checks for duplicates, for each box
# it'll make a note of the canonical copy in this list.
dedup_list = []

# Let's make a metaphorical switch, and let's turn it off.
switch = 'off'

#Look through the dup_list that you produced in a previous script. . . 
with jsonlines.open('nea_containers.jsonl') as dup_list:
    # For each NEA container in that list . . . 
    for container in dup_list:
        # Save the container's box number and URL to variables.
        indicator = (container['indicator'])
        uri = (container['uri'])
        # Get a fresh copy of the container from ASpace
        container = requests.get(HOST + uri,
                                  headers=headers)
        container = container.json()
        # Ok, so for this NEA container on the list, you've got a fresh copy on hand.
        # Check to see if its box number has already been added to the dedup_list
        for d in dedup_list:
            # If has already been added, then you've found a duplicate record! 
            if d.get('indicator', None) == indicator:
                # Turn your metaphorical "switch" on 
                switch = 'on'
                # Save the canonical uri
                canonical_uri = d['uri']
                # Print status to the terminal.
                print('Duplicate: ' + uri + ', ' + indicator + ' (canonical: ' + canonical_uri + ')')
                # Don't bother looking through the rest of the list. 
                break
            #If it checks all of dedup_list and doesn't find it, don't do anything just yet.
            
        # If that box number was in dedup_list, the "switch" is on, and you want to mark the container as a duplicate. 
        if switch == 'on':
            # Edit temporarily park the canonical URI into the "ils_holding_id" field.
            container['ils_holding_id'] = canonical_uri
            edit = requests.post(HOST + uri,
                             data=json.dumps(container),
                             headers=headers)
            print('Parking canonical URI in ILS Holding ID field. Response should be 200 if edit worked.')
            print(edit)
            result = requests.get(HOST + uri,
                                  headers=headers)
        else:
            add_this = {'indicator':indicator, 'uri': uri}
            dedup_list.append(add_this)
            print('NEW: ' + uri + ', ' + indicator)
            print('Added to dedup_list as canonical record')
        print('')
        #turn the switch back off
        switch = 'off'